# GNU Scientific Library & Gnuplot(Ubuntu)

https://www.gnu.org/software/gsl/

The GNU Scientific Library (GSL) is a numerical library for C and C++ programmers. It is free software under the GNU General Public License.

The library provides a wide range of mathematical routines such as random number generators, special functions and least-squares fitting. There are over 1000 functions in total with an extensive test suite.


## 1 Install GSL under Ubuntu

**Ubuntu Packages Search**

   This site provides you with information about all the packages available in the Ubuntu Package archive.

   https://packages.ubuntu.com/

Ubuntu 18.04 https://packages.ubuntu.com/bionic/math/gsl-bin

* gsl-2.3 was released in December 2016

Ubuntu 16.04 https://packages.ubuntu.com/xenial/math/gsl-bin

* gsl-2.1 was released in November 2015

**Installing GSL**

```bash
$sudo apt install gsl-bin

$sudo apt install libgsl-dev
```

## 2 Using the GSL Library

### 2.1 Compiling and Linking

The library header files are installed in their own gsl directory. You should write any preprocessor include statements with a gsl/ directory prefix thus, 

```c
#include <gsl/gsl_rstat.h>
```
If the directory is not installed on the standard search path of your compiler you will also need to provide its location to the preprocessor as a command line flag.The default location of the gsl directory(ubuntu18.04) is `/usr/include/gsl`

A typical compilation command for a source file example.c with the GNU C compiler gcc is, 

```bash
$gcc -Wall -I/usr/include -c example.c
```
This results in an object file example.o. The default include path for gcc searches `/usr/include` automatically so the `-I` option can actually be **omitted** when GSL is installed in its default location. 

```bash
$gcc -Wall -c example.c
```


### 2.2 Linking programs with the library

The library is installed as a single file, `libgsl.a`. A shared version of the library `libgsl.so` is also installed on systems that support shared libraries. The default location of these files(Ubuntu18.04) is `/usr/lib/x86_64-linux-gnu/`. If this directory is not on the standard search path of your linker you will also need to provide its location as a command line flag.

To link against the library you need to specify both the `main library` and a supporting` CBLAS library,` which provides standard basic linear algebra subroutines. A suitable CBLAS implementation is provided in the library `libgslcblas.a` if your system does not provide one. The following example shows how to link an application with the library:

```bash
$gcc -L/usr/lib/x86_64-linux-gnu/ example.o -lgsl -lgslcblas -lm
```
The default library path for gcc searches `/usr/lib/x86_64-linux-gnu/` automatically so the `-L` option can be **omitted** when GSL is installed in its default location.

The option `-lm` links with the system math library. 


### 2.3 The  Example 

Here is a basic example of how to use the statistical functions

https://www.gnu.org/software/gsl/doc/html/rstat.html

In [ ]:
%%file ./code/gcc/example.c

#include <stdio.h>
#include <gsl/gsl_rstat.h>

int main(void)
{
  double data[5] = {17.2, 18.1, 16.5, 18.3, 12.6};
  
  double mean, variance, largest, smallest, sd,
         rms, sd_mean, median, skew, kurtosis;
    
  gsl_rstat_workspace *rstat_p = gsl_rstat_alloc();
  size_t i, n;

  /* add data to rstat accumulator */
  for (i = 0; i < 5; ++i)
    gsl_rstat_add(data[i], rstat_p);

  mean     = gsl_rstat_mean(rstat_p);
  variance = gsl_rstat_variance(rstat_p);
  largest  = gsl_rstat_max(rstat_p);
  smallest = gsl_rstat_min(rstat_p);
  median   = gsl_rstat_median(rstat_p);
  sd       = gsl_rstat_sd(rstat_p);
  sd_mean  = gsl_rstat_sd_mean(rstat_p);
  skew     = gsl_rstat_skew(rstat_p);
  rms      = gsl_rstat_rms(rstat_p);
  kurtosis = gsl_rstat_kurtosis(rstat_p);
  n        = gsl_rstat_n(rstat_p);

  printf ("The dataset is %g, %g, %g, %g, %g\n",
         data[0], data[1], data[2], data[3], data[4]);

  printf ("The sample mean is %g\n", mean);
  printf ("The estimated variance is %g\n", variance);
  printf ("The largest value is %g\n", largest);
  printf ("The smallest value is %g\n", smallest);
  printf( "The median is %g\n", median);
  printf( "The standard deviation is %g\n", sd);
  printf( "The root mean square is %g\n", rms);
  printf( "The standard devation of the mean is %g\n", sd_mean);
  printf( "The skew is %g\n", skew);
  printf( "The kurtosis %g\n", kurtosis);
  printf( "There are %zu items in the accumulator\n", n);

  gsl_rstat_reset(rstat_p);
  n = gsl_rstat_n(rstat_p);
  printf( "There are %zu items in the accumulator\n", n);

  gsl_rstat_free(rstat_p);

  return 0;
}

In [ ]:
!gcc -Wall -I/usr/include -c ./code/gcc/example.c

In [ ]:
!gcc -o example example.o -L/usr/lib/x86_64-linux-gnu/ -lgsl -lgslcblas -lm

In [ ]:
!./example

**-I -L** option can be **omitted** when GSL is installed in its default location

In [ ]:
!gcc -Wall -c ./code/gcc/example.c

In [ ]:
!gcc -o example example.o -lgsl -lgslcblas -lm

In [ ]:
!./example 

## 3 Using GSL - Linear Least-Squares Fitting

The functions described in this section can be used to perform least-squares fits to a straight line model, .

```c
int gsl_fit_linear(const double * x, const size_t xstride, const double * y, const size_t ystride, size_t n, double * c0, double * c1, double * cov00, double * cov01, double * cov11, double * sumsq)
```
This function computes the best-fit linear regression coefficients `(c0, c1)` of the model  for the dataset `(x, y)`, two vectors of length `n` with strides xstride and ystride.

The errors on `y` are assumed unknown so the variance-covariance matrix for the parameters `(c0, c1)` is estimated from the scatter of the points around the best-fit line and returned via the parameters` (cov00, cov01, cov11)`. 

The **sum of squares of the residuals** from the best-fit line is returned in `sumsq`. 

The following program computes a least squares straight-line fit to one dataset of `Mr. F.J. Anscombe`, and outputs the best-fit line and its associated one standard-deviation error bars.

In [ ]:
%%file ./code/gcc/demo_fit.c

#include <stdio.h>
#include <gsl/gsl_fit.h>

int main (void)
{
  int n = 11;
  double x[11] = {10.0, 8.0, 13.0, 9.0,11.0,14.0,6.0,4.0 ,12.0,7.0,5.0};
  double y[11] = {  8.04,  6.95,7.68, 8.81, 8.33,9.96,7.24,4.26,10.84,4.82,5.68 };

  double c0, c1, cov00, cov01, cov11, sumsq;

  gsl_fit_linear (x, 1, y, 1, n,
                   &c0, &c1, &cov00, &cov01, &cov11,
                   &sumsq);

  printf ("best fit: Y = %g + %g X\n", c0, c1);
  printf ("covariance matrix:\n");
  printf ("[ %g, %g\n  %g, %g]\n",
          cov00, cov01, cov01, cov11);
  printf ("sumsq = %g\n", sumsq);

  printf ("\n");

  return 0;
}

In [ ]:
!gcc -o  demo_fit ./code/gcc/demo_fit.c -lgsl -lgslcblas -lm

In [ ]:
!./demo_fit 

## 4 Gnuplot 

http://gnuplot.info/
   
Gnuplot is a portable command-line driven graphing utility for Linux, OS/2, MS Windows, OSX, VMS, and many other platforms.

The source code is copyrighted but freely distributed (i.e., you don't have to pay for it). It was originally created to allow scientists and students to visualize mathematical functions and data interactively, but has grown to support many non-interactive uses such as web scripting. 

It is also used as a plotting engine by third-party applications like Octave. Gnuplot has been supported and under active development since 1986.

**Installing**

```
$sudo apt install gnuplot
```

### Plotting Data with pipe(popen)

#### 15.2 Pipe to a Subprocess

Using a **pipe** to **communicate** with a **child** proces

https://www.gnu.org/software/libc/manual/html_node/Pipe-to-a-Subprocess.html

**popen**

```c
#include <stdio.h>
FILE *popen(const char *command, const char *mode); 
```

DESCRIPTION

The popen() function shall execute the command specified by the string command. It shall create a pipe between the calling program and the executed command, and shall return a pointer to a stream that can be used to either read from or write to the pipe.

**pipe**

A pipe is a mechanism for interprocess communication; data written to the pipe by one process can be read by another process.

The pipe is used to **transfer** the **results** from **one command to another**.

**subprocess**

A Linux **process** is a program running in the Linux system.

A **subprocess** is a child process, i.e. a process that has been launched by its parent to which it is a subprocess.

#### Example 1 data file

Discrete data contained in a file can be displayed by specifying the name of the data file (enclosed in quotes) on the plot or splot command line. 

**Data files** should have the data arranged in `columns` of numbers. Columns should be separated by `datafile separator`.

`Lines` beginning with a `#` character are treated as `comments` and are ignored by Gnuplot. 

A `blank` line in the data file results in a break in the line connecting data points.


In [ ]:
%%file ./src/springData.csv
#Distance(m),Mass(kg)
0.0865,0.1
0.1015,0.15
0.1106,0.2
0.1279,0.25
0.1892,0.3
0.2695,0.35
0.2888,0.4
0.2425,0.45
0.3465,0.5
0.3225,0.55
0.3764,0.6
0.4263,0.65
0.4562,0.7
0.4502,0.75
0.4499,0.8
0.4534,0.85
0.4416,0.9
0.4304,0.95
0.437,1.0

In [ ]:
%%file ./code/gcc/pipegnuplot_datafile.c

#include <stdio.h>

int main (void)
{

    FILE *gnuplotPipe = popen("gnuplot -persist", "w");  // Open a pipe to gnuplot

    if (gnuplotPipe) {   // If gnuplot is found
      fprintf(gnuplotPipe, "set datafile separator ','\n"); //datafile separator ','
      fprintf(gnuplotPipe, "set xlabel 'Distances'\n");
      fprintf(gnuplotPipe, "set ylabel 'Mass'\n");
      fprintf(gnuplotPipe, "set title 'Observable'\n");
      fprintf(gnuplotPipe, "plot './src/springData.csv' using 1:2 title 'Mass=f(Distance)-' \n");

    fflush(gnuplotPipe); //flush pipe

    fprintf(gnuplotPipe,"exit \n");   // exit gnuplot
    pclose(gnuplotPipe);    //close pipe
    }
    return 0;
};

In [ ]:
!gcc  -o pipegnuplot_datafile ./code/gcc/pipegnuplot_datafile.c

In [ ]:
!./pipegnuplot_datafile

#### Example 2  plot variables

In [ ]:
%%file ./code/gcc/pipegnuplot_variables.c

#include <stdio.h>
#include <gsl/gsl_fit.h>

int main (void)
{
  int n = 11;
  double x[11] = {10.0, 8.0, 13.0, 9.0,11.0,14.0,6.0,4.0 ,12.0,7.0,5.0};
  double y[11] = {8.04, 6.95,7.68, 8.81, 8.33,9.96,7.24,4.26,10.84,4.82,5.68 };

  double c0, c1, cov00, cov01, cov11, sumsq;

  gsl_fit_linear(x, 1, y, 1, n,
                   &c0, &c1, &cov00, &cov01, &cov11,
                   &sumsq);

  printf ("best fit: Y = %g + %g X\n", c0, c1);
  printf ("covariance matrix:\n");
  printf ("[ %g, %g\n  %g, %g]\n",
          cov00, cov01, cov01, cov11);
  printf ("sumsq = %g\n", sumsq);
  printf ("\n");
  
  // plot
  FILE *gnuplotPipe = popen("gnuplot -persist", "w"); // Open a pipe to gnuplot
  if (gnuplotPipe) // If gnuplot is found
  { 
     fprintf(gnuplotPipe, "set xlabel 'X'\n");
     fprintf(gnuplotPipe, "set ylabel 'Y'\n");
     fprintf(gnuplotPipe, "set title '<X,Y> and Linear fit'\n");
      
     /* 
        1 sending gnuplot the plot '-' command 
        2 followed by data points 
        3 followed by the letter "e" 
     */
     
     // 1  sending gnuplot the plot '-' command
     fprintf(gnuplotPipe, "plot '-' title '<x,y>','-' title 'Line' with line ls 12 \n");
     
     // 2 followed by data points: <x,y>
     for (int i = 0; i < n; i++)
     {
        fprintf(gnuplotPipe, "%lf %lf\n", x[i], y[i]);
     }
     // 3 followed by the letter "e" 
     fprintf(gnuplotPipe, "e");
     
     // linear fit
     for (int i = 0; i < n; i++)
     {
        fprintf(gnuplotPipe, "%lf %lf\n", x[i], c0+c1*x[i]);
     }
     fprintf(gnuplotPipe, "e");
      
     fflush(gnuplotPipe);
     fprintf(gnuplotPipe, "exit \n"); // exit gnuplot
     pclose(gnuplotPipe);             //close pipe
  }
  
  return 0;
}

In [ ]:
!gcc  -o pipegnuplot_variables ./code/gcc/pipegnuplot_variables.c -lgsl -lgslcblas -lm 

In [ ]:
!./pipegnuplot_variables

# Reference

* GSL - GNU Scientific Library https://www.gnu.org/software/gsl/


* The GSL Reference Manual online https://www.gnu.org/software/gsl/doc/html/index.html


* Gnuplot homepage http://gnuplot.info/


* Nishanth Sastry：gnuplot 让您的数据可视化 https://www.ibm.com/developerworks/cn/linux/l-gnuplot/


* The GNU C Library (glibc) https://www.gnu.org/software/libc/